In [1]:
#access to your google drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/data")  # file path, your path might be different
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/data


In [2]:
#import package here
!pip install tensorflow_decision_forests
import numpy as np
import pandas as pd
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn import metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing insta

In [3]:
#load training data

train_data = pd.read_csv('train.csv')

labels = train_data['label'].values
features = train_data.drop('label', axis=1).values

#split into training and validation set
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)
print(f"shape x train={X_train.shape}, x valid={X_val.shape}, y train={y_train.shape}, y valid={y_val.shape}")

shape x train=(400000, 35), x valid=(100000, 35), y train=(400000,), y valid=(100000,)


In [4]:
#training

#X_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(X_train, label="label")
model = tfdf.keras.RandomForestModel()
model.fit(X_train, y_train)

Use /tmp/tmpjuv6m29t as temporary training directory
Reading training dataset...
Training dataset read in 0:00:17.194222. Found 400000 examples.
Training model...
Model trained in 0:14:41.907822
Compiling model...
Model compiled.


In [5]:
#validation

y_pred=model.predict(X_val)
y_pred=np.reshape(y_pred, -1)
print(y_pred[:10])

#find the threshold that has the best f1 score
max_score=-1
best_ratio=-1
for ratio in range(0,100):  #ratio = [0, 1, 2, ..., 99]
    tmp=[]
    for idx in range(len(y_pred)):
        if y_pred[idx]>=ratio/100:  #ratio/100 = [0, 0.01, 0.02, ..., 0.99]
            tmp.append(1)
        else:
            tmp.append(0)

    score=metrics.f1_score(y_val,tmp)   #y_val is ground truth
    if score>max_score:
        max_score=score
        best_ratio=ratio/100
print(f"ratio={best_ratio}, f1 score = {max_score}")

3125/3125 [==============================] - 18s 6ms/step
[0.00666667 0.03       0.52333295 0.         0.         0.
 0.         0.         0.02666667 0.04333334]
ratio=0.24, f1 score = 0.6500026367136001


In [6]:
#load test data & predict

origin_data = pd.read_csv('test.csv')
test_data=origin_data.drop('id', axis=1).values
print(test_data.shape)
#test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data)
test_result = model.predict(test_data)

(200000, 35)
6250/6250 [==============================] - 42s 7ms/step


In [7]:
#convert floating point numbers to 0 or 1
print(type(test_result))
for idx in range(len(test_result)):
    if test_result[idx]>=0.24:  #best threshold = 0.24 yields max f1 score
        test_result[idx]=1
    else:
        test_result[idx]=0
test_result

<class 'numpy.ndarray'>


array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [0.]], dtype=float32)

In [8]:
#reshape
test_result=test_result.reshape(-1)
print(test_result)

[0. 0. 0. ... 1. 1. 0.]


In [9]:
#write output
with open("output.csv",'w') as fh:
    fh.write('id,label\n')
    for idx in range(len(test_result)):
        fh.write(f'{idx},{int(test_result[idx])}\n')


In [10]:
'''
refs:
https://www.tensorflow.org/decision_forests
https://www.tensorflow.org/decision_forests/tutorials
https://www.tensorflow.org/decision_forests/tutorials/beginner_colab
https://blog.tensorflow.org/2021/05/introducing-tensorflow-decision-forests.html
'''

'\nrefs:\nhttps://www.tensorflow.org/decision_forests\nhttps://www.tensorflow.org/decision_forests/tutorials\nhttps://www.tensorflow.org/decision_forests/tutorials/beginner_colab\nhttps://blog.tensorflow.org/2021/05/introducing-tensorflow-decision-forests.html\n'